In [36]:
import os, sys
from PIL import Image
import random
import numpy as np
from scipy.misc import imsave

In [39]:
#apply transformations on a single image
def binarize_image(img_path, target_path, threshold=249):
    """Binarize an image."""
    image = Image.open(img_path)
    #image = image_file.convert('L')  # convert image to monochrome
    image = np.array(image)
   
    #print(image)
    #print(set(image.flatten()))
    image = binarize_array(image, threshold)
    #print(unique_count(image))
    imsave(target_path, image)
def unique_count(x):
    unique, counts = np.unique(x, return_counts=True)
    print (np.asarray((unique, counts)).T)
def binarize_array(numpy_array, threshold=0):
    """Binarize a numpy array."""
    for i in range(len(numpy_array)):
        for j in range(len(numpy_array[0])):
            if numpy_array[i][j] >= threshold:
                #label almost white pixels as class 1, and black as 0
                numpy_array[i][j] = 1
            else:
                numpy_array[i][j] = 0
    return numpy_array

In [44]:
path = "/Users/q453991/Downloads/Archiv"
if not os.path.exists(path+"/MergedMask_annotated/"):
        os.makedirs(path+"/MergedMask_annotated/")
   
for pic in os.listdir(path+"/MergedMask"):
        if pic.endswith(".png"): 
            binarize_image(path+"/MergedMask/"+pic,path+"/MergedMask_annotated/"+pic)
           
    
   


/Users/q453991/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if sys.path[0] == '':


In [76]:
#flipping logic
def flip_image(image_path, saved_location):
    """
    Flip or mirror the image
 
    @param image_path: The path to the image to edit
    @param saved_location: Path to save the cropped image
    """
    image_obj = Image.open(image_path)
    rotated_image = image_obj.transpose(Image.FLIP_LEFT_RIGHT)
    rotated_image.save(saved_location)
    #rotated_image.show()

def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col= image.shape
        mean = 0
        var = 1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col))
        print(gauss)
        gauss = gauss.reshape(row,col)
        noisy = image + gauss
        #print(set(gauss.flatten()))
        return noisy
    elif noise_typ == "s&p":
        #row,col,ch = image.shape
        row,col = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1 - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy


In [46]:
path = "/Users/q453991/Downloads/Archiv"
for item in ["MergedMask","ResizedCXR"]:
    if not os.path.exists(path+"/"+item+"_flipped/"):
            os.makedirs(path+"/"+item+"_flipped/")

    for pic in os.listdir(path+"/"+item):
            if pic.endswith(".png"): 
                flip_image(path+"/"+item+"/"+pic,path+"/"+item+"_flipped"+"/"+"flipped_"+pic)
#flipping logic

In [66]:
# add salt and pepper noise to original images, but also create unaltered copies of masks
path = "/Users/q453991/Downloads/Archiv"
for item in ["MergedMask","ResizedCXR"]:
    if not os.path.exists(path+"/"+item+"_saltPepper/"):
            os.makedirs(path+"/"+item+"_saltPepper/")

    for pic in os.listdir(path+"/"+item):
            if pic.endswith(".png"): 
                image_path = path+"/"+item+"/"+pic
                save_path = path+"/"+item+"_saltPepper"+"/"+"saltPepper_"+pic
                image = Image.open(image_path)
                if item is not "MergedMask":
                    image = np.array(image)
                    image = noisy(noise_typ = "s&p", image = image)
                    imsave(save_path, image)
                else:
                    image.save(save_path)
                    
#flipping logic

/Users/q453991/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  from ipykernel import kernelapp as app


In [77]:
# add salt and pepper noise to original images, but also create unaltered copies of masks
path = "/Users/q453991/Downloads/Archiv"
for item in ["MergedMask","ResizedCXR"]:
    if not os.path.exists(path+"/"+item+"_gaussianNoise/"):
            os.makedirs(path+"/"+item+"_gaussianNoise/")

    for pic in os.listdir(path+"/"+item):
            if pic.endswith(".png"): 
                image_path = path+"/"+item+"/"+pic
                save_path = path+"/"+item+"_gaussianNoise"+"/"+"gaussianNoise_"+pic
                image = Image.open(image_path)
                if item is not "MergedMask":
                    image = np.array(image)
                    image = noisy(noise_typ = "gauss", image = image)
                    imsave(save_path, image)
                else:
                    image.save(save_path)
                    
#flipping logic



[[ 0.42406017  2.14304617  0.44567565 ... -0.14672382 -0.26683876
   1.38771781]
 [-0.94449273 -0.1741901   1.3027761  ...  0.85179965 -1.20148405
  -0.63982279]
 [ 0.81298443  2.02245702 -1.22157959 ... -1.67980966  0.7126188
   0.03749742]
 ...
 [ 2.05487681 -0.33419899  0.9013822  ...  0.25038693 -0.07861034
  -0.49849001]
 [-1.513398   -0.96431659  0.94620959 ...  0.75338062  2.03666047
  -0.46502698]
 [-1.25021668 -2.02077398  0.17802776 ... -1.83818686  3.01088509
  -0.34908098]]
[[ 0.92428618  0.93947388 -0.11522824 ...  0.14900145  0.46318641
   0.4304859 ]
 [-0.01379747  0.06447142  1.4222561  ...  1.40338605 -0.24278477
   0.66765732]
 [-0.63925469 -0.3982948  -1.21734963 ...  0.24732225  0.25320587
  -0.58981781]
 ...
 [-0.03917462 -0.41307692  0.76813205 ... -1.79065448  0.33477289
   1.42359698]
 [ 0.11256707  1.21550361  1.35390861 ... -0.86492545  2.63514777
   0.52752725]
 [-1.11888583  0.57413305 -0.24997184 ... -0.54789671 -1.46490385
   1.17676638]]
[[ 0.53904991  0.

/Users/q453991/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  from ipykernel import kernelapp as app


[[ 1.30605646 -0.31620183  0.23946169 ... -0.5309417  -0.12686417
  -0.39568376]
 [-0.06407545  1.36676444 -0.27858257 ...  0.04981567 -1.659077
  -1.04409448]
 [ 0.94383543 -1.7180822  -0.24012714 ...  0.59535624 -0.56689261
   0.51678351]
 ...
 [ 0.2648238   0.81466471 -0.21687739 ...  0.11609449 -2.14568978
   1.16187896]
 [ 1.50986447  0.6377853   2.25675033 ... -0.09514338 -0.64533109
   0.54110918]
 [-0.10559959 -1.49903713  1.49321867 ...  0.81627965  1.16622541
   1.98873541]]
[[ 0.34328638 -1.22439504  0.86511609 ... -0.96133391  0.30918469
   1.30164452]
 [ 0.96466879  2.04928119 -1.98658279 ...  0.29963908 -0.30653411
   0.5740509 ]
 [ 0.33428247  0.57602534  0.16203007 ...  1.2020874  -0.24122057
  -0.02658123]
 ...
 [-0.71562776 -0.96937565  0.0859373  ... -1.17643384  1.02186684
  -0.77376558]
 [-0.50196822 -1.04801842  0.59454418 ... -0.30194875  1.05110912
  -0.27539352]
 [-1.40455586  1.19542007 -0.24111479 ... -1.43409816  0.39937676
  -0.06669578]]
[[ 0.54719781  1.0

[[ 1.23873918 -0.09805941  0.17144434 ...  1.64364632 -0.14394653
   0.62945871]
 [ 0.00530729  1.94300542  0.55837136 ...  0.6487399  -0.94201084
   1.52978532]
 [ 2.53278157 -0.42150196  0.4151514  ... -0.68752851  0.80655495
  -2.21374213]
 ...
 [ 0.49931021  0.83475467 -2.48358835 ... -2.02939051 -1.6912847
   0.69765103]
 [ 0.66729457  0.86484868 -0.02816972 ... -1.27769495 -0.97130996
  -2.06813044]
 [-0.85508055 -1.8851896   0.23865824 ...  1.52390099 -0.70859435
   0.15691881]]
[[ 0.79443752  0.5025922  -0.31511876 ...  0.61052093  0.87639847
   0.81458672]
 [ 0.50411269  0.34276696 -0.33176841 ... -0.74640599 -1.35131496
  -2.88820781]
 [-2.11522661  0.1815142   0.36389487 ...  0.5476177  -1.18286111
   1.02941354]
 ...
 [-0.59994855 -1.27876654  0.46828373 ...  0.69529373 -0.32559726
  -0.90462707]
 [ 0.53150249 -0.51800344 -0.27899972 ...  0.24559503 -1.48287579
  -1.50213421]
 [ 1.27509737 -0.8019786  -0.24089965 ... -0.35211331  1.0504889
   0.59728923]]
[[-2.10510773 -1.3

[[ 0.1707584  -0.05989808 -0.20652358 ...  0.17208658 -1.24471973
  -0.07128793]
 [ 1.04304194  0.88888918  0.29530465 ...  0.31916751  0.12005659
   0.87344165]
 [-0.34031667  0.99001092  0.7248231  ...  0.98867542  0.1941415
   0.6506688 ]
 ...
 [-1.90116251  1.32475044 -0.49234754 ... -0.07699305  0.8295363
  -0.90294531]
 [-0.02104123  0.77163251 -0.27227432 ... -0.5513789  -1.09142804
  -0.48889285]
 [ 0.51464408 -1.0654452  -0.74481993 ... -0.15235505 -0.27050977
   1.31256035]]
[[-2.37101044 -0.29702067  0.23440904 ...  0.24835727 -0.21979908
  -0.96291151]
 [ 0.71776789 -0.15159569 -1.32054541 ... -1.73069684 -0.8320993
   0.00396441]
 [ 0.83648878 -0.3767025  -1.79094586 ...  0.31090642  0.40621967
   0.56751058]
 ...
 [-0.1385437  -1.65836356  0.33787145 ... -0.5098369   0.96291634
   0.08683585]
 [-0.72887056 -0.35553809 -0.22772145 ...  1.84142514 -0.13936211
   1.2108009 ]
 [ 0.3238161   0.40078992 -0.17106206 ...  0.88228423 -1.02956102
   0.711652  ]]
[[-2.3805479   1.91

[[ 0.37462249  0.54662564  0.51972214 ... -0.14325426  1.27740099
   1.1046072 ]
 [ 1.21274284 -1.15543329  0.01324958 ... -0.97416347  0.37108319
   1.49835329]
 [ 0.6427284  -0.1708713   1.09411791 ...  1.25416662  0.18327385
  -0.375326  ]
 ...
 [ 0.82720345  1.72096532 -0.19377512 ... -0.34278391 -1.16251861
   1.36929401]
 [-0.01365524 -0.17401411  1.11575703 ...  0.18833093 -0.6749177
  -1.00252166]
 [-0.37911683  0.6329317  -0.27790047 ... -0.14013097 -1.2659684
  -0.6755032 ]]
[[-0.65676138  1.26237018  0.60323973 ... -0.45009197  1.2133169
  -1.33497812]
 [-0.63222804 -1.99691586  0.83664603 ...  1.12299036  0.27975602
   0.49880134]
 [-0.23413337  1.55102584 -0.99578317 ...  0.38452853 -1.09152833
   0.51087652]
 ...
 [ 0.00419936 -0.13457851  0.08630682 ... -0.29283546 -0.84830136
  -0.8196115 ]
 [ 0.49334215  1.52099939  0.26519939 ...  1.80834867 -1.48204277
  -0.36727158]
 [-1.10792356  1.22477582 -1.5444838  ...  0.93215576 -0.98017949
  -1.15327115]]
[[-1.49172704 -1.45

KeyboardInterrupt: 

In [78]:
import Augmentor

In [84]:
p = Augmentor.Pipeline("/Users/q453991/Downloads/Archiv copy/ResizedCXR")
# Point to a directory containing ground truth data.
# Images with the same file names will be added as ground truth data
# and augmented in parallel to the original data.
p.ground_truth("/Users/q453991/Downloads/Archiv copy/MergedMask")

Initialised with 138 image(s) found.
Output directory set to /Users/q453991/Downloads/Archiv copy/ResizedCXR/output.138 ground truth image(s) found.


In [85]:

p.random_contrast(probability=1,min_factor= .05, max_factor=.1)
p.process()

Processing <PIL.Image.Image image mode=L size=480x360 at 0x1102704E0>: 100%|██████████| 138/138 [00:01<00:00, 126.04 Samples/s]
